In [12]:
import matplotlib.pyplot as plt
import datetime
import torch
import torch.nn as nn
import torch.nn.functional as F
import copy
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torchvision.models as models

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()


In [13]:
class Net(nn.Module):
        def __init__(self):
                super(Net, self).__init__()
                self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
                self.conv2 = nn.Conv2d(32, 32, kernel_size=5)
                self.conv3 = nn.Conv2d(32,64, kernel_size=5)
                self.fc1 = nn.Linear(3*3*64, 256)
                self.fc2 = nn.Linear(256, 10)
                
        def forward(self, x):
                x = F.relu(self.conv1(x))
                x = F.relu(F.max_pool2d(self.conv2(x), 2))
                x = F.dropout(x, p=0.5, training=self.training)
                x = F.relu(F.max_pool2d(self.conv3(x),2))
                x = F.dropout(x, p=0.5, training=self.training)
                x = x.view(-1,3*3*64 )
                x = F.relu(self.fc1(x))
                x = F.dropout(x, training=self.training)
                x = self.fc2(x)
                return F.log_softmax(x, dim=1)

model = Net()
model

Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)

In [14]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs):

    train_losses = []
    valid_losses = []
    best_model_loss = 5
    
    for epoch in range(num_epochs):

        # Training
        for i, (data, labels) in enumerate(train_loader):
      
            prediction = model.forward(data)

            train_loss = criterion(prediction, labels)

            train_loss.backward()

            optimizer.step()

            optimizer.zero_grad()
        print(f'\rEpoch {epoch+1}, batch {i+1}/{len(train_loader)} - Loss: {train_loss}')

        train_losses.append(train_loss)
        writer.add_scalar("Loss/train ADAM", train_loss, epoch)

        # Validation
        for batch_nr, (data, labels) in enumerate(val_loader):
            prediction = model.forward(data)
            loss_val = criterion(prediction, labels)
            valid_losses.append(loss_val)
        print(f"loss validation: {loss_val}")

        if valid_losses[-1] < best_model_loss:
            print(f"\t > Found a better model, {best_model_loss} -> {valid_losses[-1]}")
            best_model = copy.deepcopy(model)
            best_model_loss = valid_losses[-1]

        writer.add_scalar("Loss/validation ADAM", loss_val, epoch)

    print(f"\nBest model loss: {best_model_loss}")
    return best_model, train_losses, valid_losses

def get_accuracy(network, loader):
    
    with torch.no_grad():
        correct = 0
        total = 0
        y_pred = []
        y_true = []

        for x, (data, labels) in enumerate(loader):

            prediction = network.forward(data)

            for i in range(len(data)):

                y_true.append(labels[i].item())
                y_pred.append(torch.argmax(prediction[i]).item())
                if y_true[i] == y_pred[i]:
                    correct += 1        
    
            total += float(len(data))
    
        score = correct/total

        accuracy = score

        return accuracy
    

In [15]:

LEARNING_RATE = 0.001
EPOCHS = 5
BATCH_SIZE = 1000

transform = transforms.Compose([transforms.ToTensor()])

trainset = torchvision.datasets.MNIST(root='./data', train=True,download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data', train=False,download=True, transform=transform)

validset, trainset = torch.utils.data.random_split(trainset, [10000, 50000])

trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,shuffle=True)
validloader = torch.utils.data.DataLoader(validset, batch_size=BATCH_SIZE,shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,shuffle=False)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

# Train the model
trained_model, train_loss, valid_loss = train_model(model, criterion, optimizer, trainloader, validloader, EPOCHS)

# Test the model
test_acc = get_accuracy(trained_model, testloader)
print(f"Model Accuracy (MNIST): {test_acc*100}%")
writer.flush()


Epoch 1, batch 50/50 - Loss: 0.32739126682281494
loss validation: 0.37020277976989746
	 > Found a better model, 5 -> 0.37020277976989746
Epoch 2, batch 50/50 - Loss: 0.20867928862571716
loss validation: 0.21903079748153687
	 > Found a better model, 0.37020277976989746 -> 0.21903079748153687
Epoch 3, batch 50/50 - Loss: 0.1739194095134735
loss validation: 0.179385244846344
	 > Found a better model, 0.21903079748153687 -> 0.179385244846344
Epoch 4, batch 50/50 - Loss: 0.10918772965669632
loss validation: 0.1228712722659111
	 > Found a better model, 0.179385244846344 -> 0.1228712722659111
Epoch 5, batch 50/50 - Loss: 0.09994558990001678
loss validation: 0.09697303920984268
	 > Found a better model, 0.1228712722659111 -> 0.09697303920984268

Best model loss: 0.09697303920984268
Model Accuracy (MNIST): 95.7%


### SVHN

In [16]:

transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),transforms.Grayscale(num_output_channels=1),transforms.Resize(28)])

dataset = torchvision.datasets.SVHN(root='./data',download=True,transform=transform)

testset, validset, trainset = torch.utils.data.random_split(dataset, [10000,12000,51257])

trainloader_svhn = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,shuffle=True)
validloader_svhn = torch.utils.data.DataLoader(validset, batch_size=BATCH_SIZE,shuffle=True)
testloader_svhn = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,shuffle=False)

# Test the model
test_acc = get_accuracy(trained_model, testloader_svhn)
print(f"Model Accuracy (SVHN): {test_acc*100}%")
writer.flush()

Using downloaded and verified file: ./data/train_32x32.mat
Model Accuracy (SVHN): 22.6%


### Transfer Learning

In [22]:
model_finetune = trained_model


LEARNING_RATE = 0.0001
EPOCHS = 5

# Define our loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_finetune.parameters(), lr = LEARNING_RATE)

# Train the model
trained_model_finetuned, train_loss, valid_loss = train_model(model_finetune, criterion, optimizer, trainloader_svhn, validloader_svhn, EPOCHS)

# Test the model
test_acc = get_accuracy(trained_model_finetuned, testloader_svhn)
print(f"Model Accuracy (Transfer Leraning): {test_acc*100}%")
writer.flush()

Epoch 1, batch 52/52 - Loss: 1.1429128646850586
loss validation: 1.2087353467941284
	 > Found a better model, 5 -> 1.2087353467941284
Epoch 2, batch 52/52 - Loss: 1.2255258560180664
loss validation: 1.1519882678985596
	 > Found a better model, 1.2087353467941284 -> 1.1519882678985596
Epoch 3, batch 52/52 - Loss: 1.165192723274231
loss validation: 1.1611745357513428
Epoch 4, batch 52/52 - Loss: 1.0515843629837036
loss validation: 1.178928017616272
Epoch 5, batch 52/52 - Loss: 1.098952293395996
loss validation: 1.121430516242981
	 > Found a better model, 1.1519882678985596 -> 1.121430516242981

Best model loss: 1.121430516242981
Model Accuracy (Transfer Leraning): 62.6%


## Feature Extraction

In [27]:
model_featext = trained_model

LEARNING_RATE = 0.0001
EPOCHS = 5

# Freeze all layers except the last few layers
for name, param in model_featext.named_parameters():
    if "fc2" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

# Define our loss function
criterion = nn.CrossEntropyLoss()

# Define our optimixer
optimizer = torch.optim.Adam(model_featext.parameters(), lr = LEARNING_RATE)

# Train the model
trained_model_featexted, train_loss, valid_loss = train_model(model_featext, criterion, optimizer, trainloader_svhn, validloader_svhn, EPOCHS)

# Test the model
test_acc = get_accuracy(trained_model_featexted, testloader_svhn)
print(f"Model Accuracy (Feature Extraction): {test_acc*100}%")
writer.flush()

Epoch 1, batch 52/52 - Loss: 1.1462393999099731
loss validation: 1.1520280838012695
	 > Found a better model, 5 -> 1.1520280838012695
Epoch 2, batch 52/52 - Loss: 1.271073579788208
loss validation: 1.1316075325012207
	 > Found a better model, 1.1520280838012695 -> 1.1316075325012207
Epoch 3, batch 52/52 - Loss: 1.1436316967010498
loss validation: 1.1628005504608154
Epoch 4, batch 52/52 - Loss: 1.1388542652130127
loss validation: 1.1367510557174683
Epoch 5, batch 52/52 - Loss: 1.1298797130584717
loss validation: 1.184921145439148

Best model loss: 1.1316075325012207
Model Accuracy (Feature Extraction): 61.9%
